In [10]:
!pip install prettytable


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [32]:
import os
import torch as th
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from envs.bidding import BiddingEnv  # Assuming this is your custom environment
from tabulate import tabulate
import numpy as np
# from prettytable import PrettyTable

In [33]:
log_dir = "./runs/baselines"


env = BiddingEnv(render_mode='human')

# Wrap with VecNormalize for observation and reward normalization
vec_env = DummyVecEnv([lambda: env])  # Single process
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)


# Load the pretrained model
model = SAC.load("./runs/baselines/SAC/best_model", env=vec_env)

In [40]:
# Reset the environment
obs = vec_env.reset()

print(obs['robot_0_self_state'])
# print(obs.robot_1_self_state)

# for item in obs:
#     if 'self_state' in item: 
#         print('self_state = ', item)

# Run for a specified number of episodes or steps
for episode in range(1):
    done = False
    total_reward = 0

    counter = 0
    
    while not done:

        counter += 1

        print(obs['robot_0_self_state'])

        # Get action from the model
        action, _ = model.predict(obs, deterministic=True)

        action_array = np.array(action).reshape(9, 6)

        # print(f"Step {counter} has Action:")
        # table = tabulate(action_array, headers=[f"Robot {i+1}" for i in range(6)], tablefmt="fancy_grid", showindex="always")
        # print(table)


        # Step the environment
        obs, reward, done, info = vec_env.step(action)
        
        # Render the environment (if your environment supports rendering)
        vec_env.render(mode='human')
        
        total_reward += reward
    
    print(f"Episode {episode + 1} Total Reward: {total_reward}")

# Close the environment when done
vec_env.close()


[[ 1.0730231 -1.1366774  1.185443 ]]
[[ 1.0730231 -1.1366774  1.185443 ]]
[[ 1.0561998 -1.1177198  1.1647209]]
[[ 1.0401437 -1.0996803  1.1450487]]
[[ 1.0247983 -1.0824869  1.1263409]]
[[ 1.0101126 -1.0660754  1.108521 ]]
[[ 0.9960407 -1.0503886  1.091521 ]]
[[ 0.98254097 -1.0353744   1.0752798 ]]
[[ 0.9695757 -1.0209862  1.0597428]]
[[ 0.95711046 -1.0071816   1.0448604 ]]
[[ 0.94511396 -0.9939223   1.0305878 ]]
Episode 1 Total Reward: [2.6313264]
